#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [34]:
DEEPINFRA_API_KEY = "JtsJG3gUjJ8TlaRJZUgFIkKYoQfoAbNS"

##0-3.Google Driveをマウント

エクセルファイルの保存先パスを設定してください。

In [35]:
G_FOLDER = "/content/drive/MyDrive/geniac"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [36]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [37]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[
        {"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
        {
            "role": "user",
            "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。",
        },
    ],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [38]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
2. 出力は指示に対する適切な応答であるべきです。出力は1か0であることを確認してください。
3. 生成するinputとoutputは全て必ず日本語で生成してください。
4. タスク指示例とは全く異なるタスクを生成してください。
5. 多様なシーンでの論理的思考力を問うことができる問題を作成してください。
6. [推論と論証, 数理的問題解決, 論理パズル, クリティカルシンキング, 論理的帰結, データ解析と解釈, 意思決定分析, 問題解決プロセス, システム思考, 倫理と論理]の中からテーマを選び、タスクを作成してください。
7. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "<設問>",
    "output": "<回答>"
  },
  {
    "id": 2,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "<設問>",
    "output": "<回答>"
  },
  {
    "id": 3,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "<設問>",
    "output": "<回答>"
  }
]
"""

In [39]:
FEWSHOT_EXAMPLES = [
    {
        "id": 1,
        "instruction": "次の論理的推論を行い、適切な結論を導いてください。",
        "input": "もしAならばB。もしBならばC。Aが真であると仮定した場合、Cは真か偽か？",
        "output": "真",
    },
    {
        "id": 2,
        "instruction": "以下のデータセットを基にした統計的分析を行い、最も適切な結論を導いてください。",
        "input": "データセット: [12, 15, 14, 10, 18, 20, 15, 14]。平均値は何でしょうか？",
        "output": "平均値は14.75です。",
    },
    {
        "id": 3,
        "instruction": "次の問題を解決するための最適な手順を考えてください。",
        "input": "5人の友人がピザを注文し、均等に分けることにしました。ピザは8枚にカットされています。友人一人あたり何枚のピザを得ることができますか？",
        "output": "友人一人あたり1枚のピザを得ることができ、3枚が余ります。",
    },
    {
        "id": 4,
        "instruction": "次の情報を分析して、最も合理的な結論を導き出してください。",
        "input": "ある工場では毎月1000個の製品を生産していますが、5%が不良品です。不良品の数を減らすためにどのような対策を講じるべきでしょうか？",
        "output": "品質管理の強化や生産プロセスの改善を検討するべきです。",
    },
    {
        "id": 5,
        "instruction": "以下のデータを解析し、適切な結論を導き出してください。",
        "input": "データセット: [20, 25, 18, 30, 22, 25, 28, 24]。中央値は何でしょうか？",
        "output": "中央値は24.5です。",
    },
    {
        "id": 6,
        "instruction": "次の問題に対する最も適切な解決策を提案してください。",
        "input": "ある会社では、従業員の満足度が低下しています。満足度を向上させるためにはどうすればよいでしょうか？",
        "output": "従業員の意見を聞き、働きやすい環境を整えることが重要です。",
    },
    {
        "id": 7,
        "instruction": "以下のシナリオに基づいて論理的帰結を導いてください。",
        "input": "もし製品Aが市場で成功するならば、製品Bも成功するでしょう。製品Aが市場で失敗しました。この場合、製品Bはどうなるでしょうか？",
        "output": "製品Bも成功しない可能性が高いです。",
    },
    {
        "id": 8,
        "instruction": "次の問題を解決するために、論理的なプロセスを提案してください。",
        "input": "Aさんは、BさんとCさんに会うために15時に出発します。Bさんとの会議は1時間、Cさんとの会議は2時間です。Aさんが帰宅するのは何時でしょうか？",
        "output": "Aさんが帰宅するのは18時です。",
    },
    {
        "id": 9,
        "instruction": "以下のシナリオを解析し、最適な行動を決定してください。",
        "input": "あるプロジェクトチームは、予算の20%を既に使い果たしました。残りの予算でどのようにプロジェクトを進行させるべきでしょうか？",
        "output": "残りの予算を慎重に管理し、優先順位を設定して進行させるべきです。",
    },
    {
        "id": 10,
        "instruction": "次のデータに基づいて、適切な結論を導いてください。",
        "input": "データセット: [45, 38, 50, 47, 42]。範囲（最大値 - 最小値）は何ですか？",
        "output": "範囲は12です。",
    },
]

##1-2.データ生成処理の実行

パラメーターをセット

In [40]:
TEMPERATURE = 1  # temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS = 10000
LOOP_NUM = 100  # 1回あたり10件データを生成

In [41]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[
            {
                "role": "system",
                "content": "あなたは日本語の学習データセットを生成する専門家です。",
            },
            {"role": "user", "content": PROMPT_TEMPLATE},
        ],
        temperature=TEMPERATURE,  # temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,  # 0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,  # -2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3,  # -2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
    )

    output = chat_completion.choices[0].message.content

    return output

In [42]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(
    FewShotExamples=FEWSHOT_EXAMPLES
)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            # output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

 [
  {
    "id": 1,
    "instruction": "以下の仮説を検証し、妥当性を評価してください。回答はYesかNoのいずれかである必要があります。",
    "input": "もし単純なアルゴリズムで問題を解くことができたら、複雑なアルゴリズムを使う必要はありません。単純なアルゴリズムで問題を解くことができたので、複雑なアルゴリズムを使う必要はありませんか？",
    "output": "Yes"
  },
  {
    "id": 2,
    "instruction": "以下のデータセットを基にした比較的評価を行い、最も適切な結論を導いてください。",
    "input": "データセット: [80, 75, 78, 82, 85]。平均値は何でしょうか？",
    "output": "平均値は80です。"
  },
  {
    "id": 3,
    "instruction": "以下の現象について論理的に考えてください。最も適切な結論を導き出してください。",
    "input": "都会では、人口密度が高く、交通規則が厳格に守られています。都会以外では人口密度が低く、交通規則が軽視されやすい傾向にあります。次の結論は正しいでしょうか？「都会では交通事故の数が少なく、都会以外では交通事故の数が多い」",
    "output": "No"
  },
  {
    "id": 4,
    "instruction": "以下のデータセットを基にした統計的分析を行い、最も適切な結論を導いてください。",
    "input": "データセット: [5, 10, 12, 7, 9, 11]。中央値は何でしょうか？",
    "output": "中央値は9.5です。"
  },
  {
    "id": 5,
    "instruction": "以下の仮説を検証し、妥当性を評価してください。回答はYesかNoのいずれかである必要があります。",
    "input": "XとYは同じような遺伝子を持っています。しかし、環境的な違いは大きく異なります。その結果、体重や身長の違いは生物学的な原因よりも環境的な原因が大きい可能性があります。",
    "output": "Yes"
  

/var/folders/bs/wbh7rgdj2fngtr9pnqy4n_mm0000gq/T/ipykernel_72271/2650223231.py:24: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_output = pd.read_json(output)


 [
  {
    "id": 1,
    "instruction": "以下の概念を組み合わせて新たな概念を創造してください。",
    "input": "環境、自動車、情報通信",
    "output": "環境に優しい情報通信ができる自動車"
  },
  {
    "id": 2,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "すべての高校生は課外活動に参加する必要があります。Johnは高校生です。Johnは課外活動に参加する必要がありますか？",
    "output": "はい"
  },
  {
    "id": 3,
    "instruction": "以下のデータを解析し、適切な結論を導き出してください。",
    "input": "データセット: [60, 55, 70, 80, 65]。平均値は何でしょうか？",
    "output": "平均値は66です。"
  },
  {
    "id": 4,
    "instruction": "以下のシナリオを解析し、最適な行動を決定してください。",
    "input": "あるプロジェクトチームは、予算の30%を既に使い果たしました。残りの予算でどのようにプロジェクトを進行させるべきでしょうか？",
    "output": "残りの予算を慎重に管理し、優先順位を設定して進行させるべきです。"
  },
  {
    "id": 5,
    "instruction": "以下の問題を解決するために、論理的なプロセスを提案してください。",
    "input": "B社は、市場調査によって製品の需要が下落していることが判明しました。B社はどのようにすべきでしょうか？",
    "output": "市場調査結果を基に、需要が低下している製品の特徴を明らかにし、改善策を考える必要があります。"
  },
  {
    "id": 6,
    "instruction": "以下のシナリオを解析し、最適な行動を決定してください。",
    "input": "C社は、新製品の開発を始めましたが、技術力不足で進捗が

/var/folders/bs/wbh7rgdj2fngtr9pnqy4n_mm0000gq/T/ipykernel_72271/2650223231.py:24: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_output = pd.read_json(output)


 [
  {
    "id": 1,
    "instruction": "以下のシナリオから適切な推論を導き出してください。回答は「yes」または「no」である必要があります。",
    "input": "もし初期投資額が高ければ、利益が高くなります。今回の投資額は非常に高いです。利益は高くなりますか？",
    "output": "yes"
  },
  {
    "id": 2,
    "instruction": "以下のデータから推論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "データセット: [15, 18, 13, 16, 19, 14, 17]。Medianの値は何ですか？",
    "output": "16"
  },
  {
    "id": 3,
    "instruction": "以下の状況を考えて、適切な行動を提案してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "ある会社は、現在従業員の年齢が上昇しており、その影響で年長者の退職率が上昇しています。会社はこの状況を変える方法は何ですか？",
    "output": "若手者の育成や引き入れの推進、老後財政についての各種サポートの提供"
  },
  {
    "id": 4,
    "instruction": "以下のシナリオから適切な推論を導き出してください。回答は「yes」または「no」である必要があります。",
    "input": "もし毎年利益が伸びていれば、前年よりも売上高が高くなるでしょう。今回の利益は過去2年に比べて大幅に増えました。前年よりも売上高が高くなりますか？",
    "output": "yes"
  },
  {
    "id": 5,
    "instruction": "以下の問題を解決するための最適な手順を考えてください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "明日あなたの友人が集まって、ピザを注文します。友人達が好むピザの種類は6つあり、1ピース1000円です。友人達は合計3000円を持っています。友人たちがピザを食べられる組み合わせの

...

 [
  {
    "id": 1,
    "instruction": "以下のシナリオに基づいて論理的帰結を導いてください。",
    "input": "もしAがTrueである場合、BがFalseになるでしょうか？",
    "output": "False"
  },
  {
    "id": 2,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "ある学校では、生徒1人当たりの平均授業時間は20時間です。その学校では、年間に合計300時間以上授業を受ける生徒は、何割ですか？",
    "output": "75%"
  },
  {
    "id": 3,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "Aさんは、他の人との比較において成績が上位25%にあります。これは、同じテストを受けた100人の中で、成績上位25人にいることを意味しますか？",
    "output": "True"
  },
  {
    "id": 4,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "グループAには男性が2人、女性が3人います。グループBには男性が3人、女性が2人います。グループAとグループBを合わせた場合、女性は男性より多くいますか？",
    "output": "True"
  },
  {
    "id": 5,
    "instruction": "以下の前提から正当な結論を導き出してください。回答は簡潔にまとめ、直接的な関連を示すようにしてください。",
    "input": "一般的に、人口が増えるにつれて、住宅の需要が上昇する傾向があります。最近の都市開発計画では、住宅の供給を増やすためにどのような取り組みが必要でしょうか？",
    "output": "新しい住宅建設計画を立案する必要があります。"
  },
  {
    "id"

/var/folders/bs/wbh7rgdj2fngtr9pnqy4n_mm0000gq/T/ipykernel_72271/2650223231.py:24: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_output = pd.read_json(output)


 [
  {
    "id": 1,
    "instruction": "以下の情報を基に、論理的な結論を導き出してください。",
    "input": "XYZ財団は、来年度の予算の５％を公益活動に割き、今年度は予算を用いずに公益活動を実施しました。来年度の予算を最大限に活用したいと考えています。今年度の方法を再度採用するか、新たな手段を取るべきでしょうか？",
    "output": "今年度の方法を再度採用し、来年度の予算を公益活動以外の活動に使用することが考えられます。"
  },
  {
    "id": 2,
    "instruction": "次の問題を解決するための手順を論理的に提案してください。",
    "input": "ある街では、交通量が多すぎて渋滞が発生しています。道路の拡張やバス及び電車の増設などを検討しています。最も効果的な解決策は何でしょうか？",
    "output": "バス及び電車の増設が最も効果的であると考えられます。"
  },
  {
    "id": 3,
    "instruction": "以下の情報を分析して、適切な結論を導き出してください。",
    "input": "ある病院では、担当医と患者の会話時間が20分あります。現在、平均して20人の患者が1日に診察されています。 AverageHandleTime（AHT）を短縮する方法は何か？",
    "output": "医師の従事業務をほかの医療従事者に代行させる方法が考えられます。"
  },
  {
    "id": 4,
    "instruction": "以下の情報を基に、適切な推論を導き出してください。",
    "input": "ある病院では、担当医と患者の会話時間が20分あります。現在、平均して20人の患者が1日に診察されています。 AverageHandleTime（AHT）を短縮することで、患者数を増やすことは可能でしょうか？",
    "output": "短縮する方法があれば、患者数を増やすことが可能です。"
  },
  {
    "id": 5,
    "instruction": "以下の問題に対する最適な解決策を提案してください。",
    "input": "ある会社では、

/var/folders/bs/wbh7rgdj2fngtr9pnqy4n_mm0000gq/T/ipykernel_72271/2650223231.py:24: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_output = pd.read_json(output)


,id,instruction,input,output,data
0,1,以下の仮説を検証し、妥当性を評価してください。回答はYesかNoのいずれかである必要があります。,もし単純なアルゴリズムで問題を解くことができたら、複雑なアルゴリズムを使う必要はありません。...,Yes,NaN
1,2,以下のデータセットを基にした比較的評価を行い、最も適切な結論を導いてください。,"データセット: [80, 75, 78, 82, 85]。平均値は何でしょうか？",平均値は80です。,NaN
2,3,以下の現象について論理的に考えてください。最も適切な結論を導き出してください。,都会では、人口密度が高く、交通規則が厳格に守られています。都会以外では人口密度が低く、交通規...,No,NaN
3,4,以下のデータセットを基にした統計的分析を行い、最も適切な結論を導いてください。,"データセット: [5, 10, 12, 7, 9, 11]。中央値は何でしょうか？",中央値は9.5です。,NaN
4,5,以下の仮説を検証し、妥当性を評価してください。回答はYesかNoのいずれかである必要があります。,XとYは同じような遺伝子を持っています。しかし、環境的な違いは大きく異なります。その結果、体...,Yes,NaN


##1-3.データの保存

In [43]:
df_final = pd.concat(df_list)

In [44]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/v1_generated_data_論理的推論_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel("output" + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /v1_generated_data_論理的推論_2024-05-17_18-48-22.xlsx
